# Cours "Géomatique" - Folium
### Louis Maritaud
### louis.maritaud@unilim.fr

## Objectifs pédagogiques
- Comprendre les différences entre cartes statiques et cartes interactives
- Créer des cartes web avec Folium
- Ajouter des marqueurs et des popups personnalisés
- Visualiser des données géographiques de manière interactive
- Exporter des cartes HTML

# Résumé de ce qu'on a vu

## GeoPandas et les GeoDataFrame
GeoPandas est une extension de Pandas qui permet de manipuler des données géographiques. Un **GeoDataFrame** est un DataFrame avec une colonne spéciale appelée `geometry` qui contient les informations géométriques (points, lignes, polygones).

**Syntaxe de base :**
```python
import geopandas as gpd

# Lire un fichier géographique
gdf = gpd.read_file("fichier.shp") # On peut lire des shapefiles ou des geojson, je préfère juste les geojson

# Voir les premières lignes
gdf.head()

# Afficher sur une carte simple
gdf.plot()
```

## Les systèmes de coordonnées (CRS)
Le **CRS** (Coordinate Reference System) définit comment les coordonnées sont interprétées :
- **EPSG:4326** (WGS84) : système latitude/longitude utilisé par le GPS
- **EPSG:2154** (Lambert-93) : système de projection pour la France métropolitaine

**Syntaxe :**
```python
# Vérifier le CRS
gdf.crs

# Reprojeter en WGS84 (nécessaire pour Folium)
gdf = gdf.to_crs(epsg=4326)
```

## Création de cartes statiques avec matplotlib
```python
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, color='blue', edgecolor='black')
plt.title("Ma carte")
plt.show()
```

# Folium, c'est quoi ?

**Folium** est une bibliothèque Python qui permet de créer des **cartes interactives** pour le web. Ces cartes peuvent être :
- Zoomées et déplacées par l'utilisateur
- Enrichies avec des popups cliquables
- Exportées en HTML pour être partagées
- Intégrées dans des sites web ou des notebooks Jupyter

**Différences avec matplotlib :**
- matplotlib → cartes **statiques** (images fixes)
- Folium → cartes **interactives** (navigation, zoom, clics)

**Folium utilise la bibliothèque JavaScript Leaflet**, ce qui permet d'avoir des cartes web professionnelles sans connaître JavaScript. En gros on écrit tout en Python, qui recompile en JS.

## Structure d'une carte Folium

Une carte Folium se construit en plusieurs étapes :
1. **Créer la carte de base** avec `folium.Map()`
2. **Ajouter des éléments** (marqueurs, polygones, etc.)
3. **Afficher ou sauvegarder** la carte

### Installation

```python
# À exécuter une seule fois dans votre terminal ou notebook
pip install folium
```

# Manipulation

In [ ]:
# Import de Folium
import folium

# Créer une carte centrée sur Limoges
# Les coordonnées sont au format [latitude, longitude]
carte = folium.Map(
    location=[45.85, 1.25],  # Centre de la carte (Limoges)
    zoom_start=13,            # Niveau de zoom initial (1=monde, 20=rue)
    tiles='OpenStreetMap'     # Fond de carte
)

# Afficher la carte dans le notebook
carte

# EXERCICE 1
Créez une carte centrée sur une ville de votre choix, et ajustez le niveau de zoom

In [ ]:
# Votre code ici


## Les différents fonds de carte (tiles)

Folium propose plusieurs fonds de carte :
- `'OpenStreetMap'` : fond par défaut, très détaillé
- `'CartoDB positron'` : fond clair et épuré
- `'CartoDB dark_matter'` : fond sombre

In [ ]:
# Carte avec un fond clair
carte_claire = folium.Map(
    location=[45.85, 1.25],
    zoom_start=12,
    tiles='CartoDB Positron'
)

carte_claire

## Ajouter des marqueurs

Les marqueurs permettent de localiser des points précis sur la carte.

In [ ]:
# Créer une carte
carte = folium.Map(
    location=[45.85, 1.25],
    zoom_start=13
)

# Ajouter un marqueur simple
folium.Marker(
    location=[45.836286250961614, 1.2674356978313563],  # Coordonnées de la gare de Limoges
    popup='Gare de Limoges-Bénédictins',  # Texte qui apparaît au clic
    tooltip='Cliquez pour plus d\'infos',  # Texte au survol
    icon=folium.Icon(color='red', icon='info-sign')  # Icône personnalisée
).add_to(carte)

# Ajouter un deuxième marqueur
folium.Marker(
    location=[45.829007591942876, 1.2659717635642285],  # Cathédrale
    popup='Cathédrale Saint-Étienne',
    icon=folium.Icon(color='blue', icon='church', prefix='fa')
).add_to(carte)

carte

# EXERCICE 2
Créez une carte avec au moins 3 marqueurs représentant des lieux importants de votre ville (université, monuments, restaurants, etc.). Utilisez des couleurs différentes pour chaque type de lieu.

In [ ]:
# Votre code ici


## Marqueurs circulaires et personnalisation

Les `CircleMarker` sont utiles pour représenter des données avec des tailles proportionnelles.

In [ ]:
import pandas as pd

# Données des principales villes du Limousin
villes = pd.DataFrame({
    'ville': ['Limoges', 'Brive-la-Gaillarde', 'Tulle', 'Guéret'],
    'latitude': [45.85, 45.1586, 45.2655, 46.1702],
    'longitude': [1.25, 1.5339, 1.7711, 1.8716],
    'population': [129754, 46697, 14723, 13315]
})

# Créer la carte
carte = folium.Map(
    location=[45.7, 1.5],
    zoom_start=9
)

# Ajouter un cercle pour chaque ville
for idx, row in villes.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['population'] / 5000,  # Taille proportionnelle à la population
        popup=f"{row['ville']}<br>Population: {row['population']:,}",
        color='crimson',
        fill=True,
        fillColor='crimson',
        fillOpacity=0.6
    ).add_to(carte)

carte

### Intégration avec GeoPandas

**Important :** Folium nécessite que les données soient en projection **EPSG:4326** (WGS84)

In [ ]:
import geopandas as gpd



# Charger les départements français
departements = gpd.read_file("DATA/departements.geojson")

# Vérifier et convertir le CRS si nécessaire
print(f"CRS actuel : {departements.crs}")

if departements.crs != 'EPSG:4326':
    departements = departements.to_crs(epsg=4326)
    print("Converti en EPSG:4326")

# Filtrer pour la Nouvelle-Aquitaine
nouvelle_aquitaine = departements[departements['nom'].isin([
    'Corrèze', 'Creuse', 'Haute-Vienne', 'Charente', 'Charente-Maritime',
    'Deux-Sèvres', 'Vienne', 'Dordogne', 'Gironde', 'Landes',
    'Lot-et-Garonne', 'Pyrénées-Atlantiques'
])]

# Créer la carte
carte = folium.Map(
    location=[45.5, 0.5],
    zoom_start=7
)

# Ajouter les polygones des départements
folium.GeoJson(
    nouvelle_aquitaine,
    name='Départements',
    style_function=lambda x: {
        'fillColor': 'lightblue',
        'color': 'darkblue',
        'weight': 2,
        'fillOpacity': 0.5
    },
    tooltip=folium.GeoJsonTooltip(fields=['nom'], aliases=['Département :'])
).add_to(carte)

carte

# EXERCICE 3 
Créez une carte qui affiche uniquement les trois départements du Limousin (Corrèze, Creuse, Haute-Vienne) avec une couleur différente pour chaque département.

In [ ]:
# Votre code ici


### Cartes choroplèthes (cartes de chaleur par zones)

Les cartes choroplèthes colorent les zones selon une valeur numérique (population, densité, etc.)

In [ ]:
# Charger les départements avec leurs populations
# (Pour l'exemple, on va ajouter des données fictives)
departements_data = nouvelle_aquitaine.copy()

# Données de population (fictives pour l'exemple)
population_dict = {
    'Corrèze': 240000,
    'Creuse': 117000,
    'Haute-Vienne': 374000,
    'Charente': 352000,
    'Charente-Maritime': 651000,
    'Deux-Sèvres': 374000,
    'Vienne': 438000,
    'Dordogne': 413000,
    'Gironde': 1600000,
    'Landes': 413000,
    'Lot-et-Garonne': 332000,
    'Pyrénées-Atlantiques': 677000
}

departements_data['population'] = departements_data['nom'].map(population_dict)

# Créer la carte choroplèthe
carte = folium.Map(
    location=[45.5, 0.5],
    zoom_start=7
)

folium.Choropleth(
    geo_data=departements_data,
    name='Population',
    data=departements_data,
    columns=['nom', 'population'],
    key_on='feature.properties.nom',
    fill_color='YlOrRd',  # Palette de couleurs (Yellow-Orange-Red)
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
).add_to(carte)

# Ajouter les tooltips
folium.GeoJson(
    departements_data,
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
    tooltip=folium.GeoJsonTooltip(
        fields=['nom', 'population'],
        aliases=['Département:', 'Population:'],
        localize=True
    )
).add_to(carte)

# Ajouter un contrôle de couches
folium.LayerControl().add_to(carte)

carte

### Carte de chaleur (HeatMap) avec points

Les HeatMap permettent de visualiser la concentration de points.

In [ ]:
from folium.plugins import HeatMap
import numpy as np

# Générer des points aléatoires autour de Limoges (pour l'exemple)
np.random.seed(42)
n_points = 200

# Coordonnées centrées sur Limoges avec un peu de variation
lats = np.random.normal(45.85, 0.05, n_points)
lons = np.random.normal(1.25, 0.05, n_points)

# Créer une liste de coordonnées
heat_data = [[lat, lon] for lat, lon in zip(lats, lons)]

# Créer la carte
carte = folium.Map(
    location=[45.85, 1.25],
    zoom_start=12
)

# Ajouter la heatmap
HeatMap(
    heat_data,
    radius=15,
    blur=25,
    max_zoom=13
).add_to(carte)

carte

# EXERCICE 4
Créez une HeatMap qui représente la localisation des restaurants ou commerces d'une ville. Vous pouvez utiliser des coordonnées fictives ou réelles si vous en avez.

In [ ]:
# Votre code ici


## Tracer des lignes et des trajets

In [ ]:
# Trajet fictif entre plusieurs villes du Limousin
trajet = [
    [45.85, 1.25],      # Limoges
    [45.1586, 1.5339],  # Brive
    [45.2655, 1.7711],  # Tulle
    [46.1702, 1.8716]   # Guéret
]

carte = folium.Map(
    location=[45.7, 1.5],
    zoom_start=9
)

# Ajouter une ligne
folium.PolyLine(
    trajet,
    color='red',
    weight=4,
    opacity=0.8,
    popup='Trajet Limousin'
).add_to(carte)

# Ajouter des marqueurs aux extrémités
for idx, point in enumerate(trajet):
    folium.Marker(
        point,
        popup=f'Étape {idx + 1}',
        icon=folium.Icon(color='red', icon='flag')
    ).add_to(carte)

carte

## Popups HTML personnalisés

Vous pouvez créer des popups avec du HTML pour un rendu plus riche.

In [ ]:
carte = folium.Map(
    location=[45.836286250961614, 1.2674356978313563],
    zoom_start=13
)

# Créer un popup HTML personnalisé
html_popup = """
<div style="font-family: Arial; width: 250px;">
    <h3 style="color: #2c5aa0;">Gare de Limoges-Bénédictins</h3>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b4/Station_Limoges-B%C3%A9n%C3%A9dictins.jpg/640px-Station_Limoges-B%C3%A9n%C3%A9dictins.jpg" 
         style="width: 100%; border-radius: 5px;">
    <p><b>Type:</b> Gare ferroviaire</p>
    <p><b>Architecture:</b> Style Art déco</p>
    <p><b>Année:</b> 1929</p>
    <p><i>Une des plus belles gares de France !</i></p>
</div>
"""

iframe = folium.IFrame(html_popup, width=270, height=350)
popup = folium.Popup(iframe, max_width=270)

folium.Marker(
    location=[45.836286250961614, 1.2674356978313563],
    popup=popup,
    icon=folium.Icon(color='blue', icon='train', prefix='fa')
).add_to(carte)

carte

# Exercice 5 :

Créez une carte interactive complète d'une région qui contient :
1. Au moins 5 marqueurs avec des popups personnalisés (HTML)
2. Un fond de carte de votre choix
3. Une couche de polygones (départements ou communes)
4. Un trajet entre plusieurs points

On sauvegardera ensuite votre carte en HTML pour pouvoir la partager.

In [ ]:
# Votre code ici


## Sauvegarder une carte en HTML

Pour partager votre carte ou l'intégrer dans un site web :

In [ ]:
# Créer une carte simple
carte = folium.Map(
    location=[45.85, 1.25],
    zoom_start=13
)

folium.Marker(
    location=[45.81772041276887, 1.23296439442943265],
    popup='Vous êtes ici',
    icon=folium.Icon(color='red', icon='star')
).add_to(carte)

# Sauvegarder en HTML
carte.save('carte_interactive.html')

print("Carte sauvegardée dans 'carte_interactive.html'")

## Plugins avancés de Folium

Folium propose de nombreux plugins pour enrichir vos cartes.

In [ ]:
from folium.plugins import MiniMap, Fullscreen, MousePosition, MeasureControl

carte = folium.Map(
    location=[45.85, 1.25],
    zoom_start=13
)

# Ajouter une mini-carte dans le coin
minimap = MiniMap(toggle_display=True)
carte.add_child(minimap)

# Bouton plein écran
Fullscreen().add_to(carte)

# Afficher les coordonnées de la souris
MousePosition().add_to(carte)

# Outil de mesure de distance
MeasureControl().add_to(carte)

carte

## Regroupement de marqueurs (MarkerCluster)

Quand vous avez beaucoup de marqueurs, MarkerCluster les regroupe automatiquement.

In [ ]:
from folium.plugins import MarkerCluster

# Générer des points aléatoires
np.random.seed(42)
n_markers = 50
lats = np.random.normal(45.85, 0.1, n_markers)
lons = np.random.normal(1.25, 0.1, n_markers)

# On créé la carte 
carte = folium.Map(
    location=[45.85, 1.25],
    zoom_start=11
)

# Créer un cluster de marqueurs
marker_cluster = MarkerCluster().add_to(carte)

# Ajouter tous les marqueurs au cluster
for lat, lon in zip(lats, lons):
    folium.Marker(
        location=[lat, lon],
        popup=f'Point ({lat:.4f}, {lon:.4f})'
    ).add_to(marker_cluster)

carte

## Récapitulatif des commandes Folium

### Créer une carte de base
```python
carte = folium.Map(
    location=[latitude, longitude],
    zoom_start=13,
    tiles='OpenStreetMap'
)
```

### Ajouter un marqueur
```python
folium.Marker(
    location=[lat, lon],
    popup='Texte au clic',
    tooltip='Texte au survol',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(carte)
```

### Ajouter un cercle
```python
folium.CircleMarker(
    location=[lat, lon],
    radius=10,
    color='blue',
    fill=True,
    fillColor='blue',
    fillOpacity=0.6
).add_to(carte)
```

### Ajouter des polygones depuis GeoPandas
```python
# Convertir en EPSG:4326 d'abord !
gdf = gdf.to_crs(epsg=4326)

folium.GeoJson(
    gdf,
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black'},
    tooltip=folium.GeoJsonTooltip(fields=['nom'])
).add_to(carte)
```

### Créer une carte choroplèthe
```python
folium.Choropleth(
    geo_data=gdf,
    data=gdf,
    columns=['nom_colonne', 'valeur_numérique'],
    key_on='feature.properties.nom_colonne',
    fill_color='YlOrRd',
    legend_name='Légende'
).add_to(carte)
```

### Sauvegarder en HTML
```python
carte.save('ma_carte.html')
```

## Ressources utiles

- **Documentation officielle Folium :** https://python-visualization.github.io/folium/
- **Exemples de cartes :** https://nbviewer.org/github/python-visualization/folium/tree/main/examples/
- **Liste des icônes Font Awesome :** https://fontawesome.com/v4/icons/
- **Palettes de couleurs :** https://colorbrewer2.org/
- **Données géographiques françaises :** https://www.data.gouv.fr/

## Conseils pour vos projets

1. **Toujours vérifier le CRS** : Folium nécessite EPSG:4326
2. **Tester avec peu de données** : Chargez d'abord un petit échantillon avant de traiter tout le dataset
3. **Utiliser MarkerCluster** : Pour plus de 20-30 marqueurs
4. **Personnaliser les popups** : Le HTML rend vos cartes plus professionnelles
5. **Optimiser les performances** : Évitez d'afficher des milliers de points individuels
